In [258]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import warnings
warnings.filterwarnings('ignore')

In [259]:
df = pd.read_csv("ecommerce_furniture_dataset_2024.csv")

In [260]:
df.head()

productTitle originalPrice    price  \
0  Dresser For Bedroom With 9 Fabric Drawers Ward...           NaN   $46.79   
1  Outdoor Conversation Set 4 Pieces Patio Furnit...           NaN  $169.72   
2  Desser For Bedroom With 7 Fabric Drawers Organ...         $78.4   $39.46   
3  Modern Accent Boucle Chair,Upholstered Tufted ...           NaN  $111.99   
4  Small Unit Simple Computer Desk Household Wood...        $48.82   $21.37   

   sold        tagText  
0   600  Free shipping  
1     0  Free shipping  
2     7  Free shipping  
3     0  Free shipping  
4     1  Free shipping

In [261]:
# Change column names
df = df.rename(columns={'productTitle': 'ProductTitle', 'originalPrice': 'OriginalPrice', 'price': 'SellingPrice', 'sold': 'UnitsSold', 'tagText': 'TagText'})

In [262]:
df.isnull().sum()

ProductTitle        0
OriginalPrice    1513
SellingPrice        0
UnitsSold           0
TagText             3
dtype: int64

In [263]:
# Dropping OriginalPrice column 
df=df.drop(['OriginalPrice'], axis=1)

In [264]:
# change null values to 0
df.fillna(0, inplace=True)

In [265]:
# In TagText column rename rows of Free shipping with Free Shipping 
df.loc[df['TagText'] == 'Free shipping', 'TagText'] = 'Free Shipping'

In [266]:
# Products where free shipping is 0 , change the 0 with Free Shipping
df.loc[df['TagText'] == 0, 'TagText'] = 'Free Shipping'

In [267]:
# Remove '$' and ',' symbols from 'SellingPrice' and 'OriginalPrice' columns
df['SellingPrice'] = df['SellingPrice'].astype(str).str.replace('$', '', regex=False).str.replace(',', '', regex=False)

# Convert 'SellingPrice' and 'OriginalPrice' to numeric
df['SellingPrice'] = pd.to_numeric(df['SellingPrice'])

In [268]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ProductTitle  2000 non-null   object 
 1   SellingPrice  2000 non-null   float64
 2   UnitsSold     2000 non-null   int64  
 3   TagText       2000 non-null   object 
dtypes: float64(1), int64(1), object(2)
memory usage: 62.6+ KB


In [269]:
df.describe()

SellingPrice     UnitsSold
count   2000.000000   2000.000000
mean     156.560020     23.493500
std      176.936735    254.094061
min        0.990000      0.000000
25%       48.530000      1.000000
50%      114.080000      3.000000
75%      193.490000      9.000000
max     2876.380000  10000.000000

In [270]:
df.shape

(2000, 4)

In [271]:
# Add new column TotalCost = sellingprice*unitsSold
df['TotalCost'] = df['SellingPrice'] * df['UnitsSold']

In [272]:
# Most expensive product in stock
df[df['SellingPrice'] == df['SellingPrice'].max()]

ProductTitle  SellingPrice  \
656  Luxury Modern Tight Curved Back Velvet Sofa,Mi...       2876.38   

     UnitsSold        TagText  TotalCost  
656          0  Free Shipping        0.0

In [273]:
# Most expensive product sold
df[df['TotalCost'] == df['TotalCost'].max()]

ProductTitle  SellingPrice  \
1008  Portable round Folding Chair Accordion Chair H...         12.28   

      UnitsSold        TagText  TotalCost  
1008      10000  Free Shipping   122800.0

In [274]:
# Least priced products list
df[df['SellingPrice'] == df['SellingPrice'].min()]

ProductTitle  SellingPrice  \
597   1PC Mini House Night Table Model Wooden Mini B...          0.99   
958   Portable Wardrobe Closet Heavy Duty Hanger Fre...          0.99   
1095  New Wholesale Simple Shoe Rack Multi-layer Spa...          0.99   
1373  1 folding chair for home and outdoor use Conve...          0.99   
1498  Office Desk Mat,Non-Slip PU Leather Desk Blott...          0.99   
1567  1pc Sequin Inflatable Sofa Colorfull Sequin La...          0.99   
1583  Card Folding Stool Portable Outdoor Camping an...          0.99   
1586  1/12 Scale mini house Miniature Round Wooden C...          0.99   

      UnitsSold        TagText  TotalCost  
597           7  Free Shipping       6.93  
958           2  Free Shipping       1.98  
1095         43  Free Shipping      42.57  
1373         85  Free Shipping      84.15  
1498         32  Free Shipping      31.68  
1567         26  Free Shipping      25.74  
1583         41  Free Shipping      40.59  
1586         23  Free Shipping      22.77

In [275]:
# Top 10 products
top_10_products = df.sort_values(by = 'UnitsSold', ascending = False).head(10)
top_10_products

ProductTitle  SellingPrice  \
1008  Portable round Folding Chair Accordion Chair H...         12.28   
1391  MUMUCC Multifunctional Foldable And Adjustable...         21.94   
696   MUMUCC Multifunctional Foldable And Adjustable...         26.11   
1021  Stylish Shoe Rack Metal Simple Shoe Shelf Foot...         13.64   
1174  Portability Foot Rest Under Desk Footrest Ergo...          8.04   
22    Office Furniture Computer Stand 42*26cm Adjust...         26.03   
1016  Monitor Stand for 13-32 inches Screens, Dual M...         66.68   
693   Portability Foot Rest Under Desk Footrest Ergo...          5.00   
1511  Portable Folding Telescopic Stool Lightweight ...         11.20   
494   Shoe-shelf Shoerack Multilayer Shoe Rack Shoeb...         15.40   

      UnitsSold        TagText  TotalCost  
1008      10000  Free Shipping   122800.0  
1391       3000  Free Shipping    65820.0  
696        3000  Free Shipping    78330.0  
1021       2000  Free Shipping    27280.0  
1174       1000  Free Shipping     8040.0  
22         1000  Free Shipping    26030.0  
1016        900  Free Shipping    60012.0  
693         900  Free Shipping     4500.0  
1511        900  Free Shipping    10080.0  
494         700  Free Shipping    10780.0

In [276]:
# Plot a bar chart to visualize the distribution
plt.figure(figsize=(8, 5))
plt.barh(top_10_products['ProductTitle'], top_10_products['UnitsSold'], color='green')
plt.xlabel('Units Sold')
plt.ylabel('Product Title')
plt.title('Top 10 Products by Units Sold')
plt.xticks(rotation=90)
plt.tight_layout()
plt.gca().invert_yaxis()
plt.show()

In [277]:
# Filtering the products within the budget of minimum $80 to maximum $500
affordable_products = df[(df['SellingPrice'] >= 80) & (df['SellingPrice'] <= 500)]
affordable_products

ProductTitle  SellingPrice  \
1     Outdoor Conversation Set 4 Pieces Patio Furnit...        169.72   
3     Modern Accent Boucle Chair,Upholstered Tufted ...        111.99   
5     3 Pieces Patio Furniture Set, Outdoor Swivel G...        123.40   
6     5-Piece Patio Furniture Set Outdoor Couch with...        198.31   
9     Outdoor Furniture Set 3 Pieces Wicker Patio Fu...        233.30   
...                                                 ...           ...   
1994  Garden Furniture 4507 (Dark Grey) Steel Frame ...        105.16   
1996  Large Wardrobe Armoire Closet with 3 Doors, Fr...        276.65   
1997  Velvet Futon Sofa Bed, 73-inch Sleeper Couch w...        196.44   
1998  Furniture Acrylic Coffee Table Transparent Liv...        228.18   
1999  Bed Frane Bamboo and Metal Platform Bed Frame ...         99.48   

      UnitsSold        TagText  TotalCost  
1             0  Free Shipping       0.00  
3             0  Free Shipping       0.00  
5            53  Free Shipping    6540.20  
6             1  Free Shipping     198.31  
9             0  Free Shipping       0.00  
...         ...            ...        ...  
1994          2  Free Shipping     210.32  
1996          2  Free Shipping     553.30  
1997         10  Free Shipping    1964.40  
1998          0  Free Shipping       0.00  
1999          0  Free Shipping       0.00  

[1152 rows x 5 columns]

In [278]:
# Replace non-numeric strings with '0' before extracting shipping charges
df['ShippingCharges'] = df['TagText'].astype(str).str.replace('Free Shipping', '0', regex=False)

# Remove '+Shipping: $' and ',' symbol from 'ShippingCharges' column
df['ShippingCharges'] = df['ShippingCharges'].str.replace('+Shipping: $', '', regex=False).str.replace(',', '', regex=False)

# Convert 'TotalCost_with_Shipping' to numeric
df['ShippingCharges'] = pd.to_numeric(df['ShippingCharges'])

# Desired order of columns
desired_column_order = ['ProductTitle', 'SellingPrice', 'UnitsSold', 'TagText', 'ShippingCharges', 'TotalCost']

# Reindex the DataFrame with the desired column order
df = df[desired_column_order]

In [279]:
# Create a new column 'TotalCost_with_Shipping'
# In the new column 'TotalCost_with_Shipping' = ShippingCharges+TotalCost , only when TagText is not free shipping and TotalCost is not 0
df['TotalCost_with_Shipping'] = np.where((df['TagText'] != 'Free Shipping') & (df['TotalCost'] != 0), df['ShippingCharges'] + df['TotalCost'], 0)